# DQA Lab

In [37]:
import os
import pandas as pd
import numpy as np

In [1]:
fp = "data/staged/calc_cases.csv"
df = pd.read_csv(fp)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1872 entries, 0 to 1871
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   patient_id                1872 non-null   object
 1   breast_density            1872 non-null   int64 
 2   left_or_right_breast      1872 non-null   object
 3   image_view                1872 non-null   object
 4   abnormality_id            1872 non-null   int64 
 5   abnormality_type          1872 non-null   object
 6   calc_type                 1848 non-null   object
 7   calc_distribution         1433 non-null   object
 8   assessment                1872 non-null   int64 
 9   pathology                 1872 non-null   object
 10  subtlety                  1872 non-null   int64 
 11  dataset                   1872 non-null   object
 12  image_series_uid          1872 non-null   object
 13  roi_mask_series_uid       1872 non-null   object
 14  cropped_image_series_uid

In [2]:
df['calc_type'].unique()

array(['AMORPHOUS', 'PLEOMORPHIC', nan,
       'ROUND_AND_REGULAR-LUCENT_CENTER-DYSTROPHIC', 'PUNCTATE', 'COARSE',
       'VASCULAR', 'FINE_LINEAR_BRANCHING', 'LARGE_RODLIKE',
       'PUNCTATE-LUCENT_CENTER',
       'VASCULAR-COARSE-LUCENT_CENTER-ROUND_AND_REGULAR-PUNCTATE',
       'ROUND_AND_REGULAR-EGGSHELL', 'PUNCTATE-PLEOMORPHIC',
       'PLEOMORPHIC-FINE_LINEAR_BRANCHING', 'DYSTROPHIC', 'LUCENT_CENTER',
       'AMORPHOUS-PLEOMORPHIC', 'ROUND_AND_REGULAR',
       'VASCULAR-COARSE-LUCENT_CENTERED', 'COARSE-ROUND_AND_REGULAR',
       'COARSE-PLEOMORPHIC', 'LUCENT_CENTERED', 'VASCULAR-COARSE',
       'ROUND_AND_REGULAR-PUNCTATE', 'ROUND_AND_REGULAR-LUCENT_CENTER',
       'COARSE-ROUND_AND_REGULAR-LUCENT_CENTERED', 'SKIN',
       'LUCENT_CENTER-PUNCTATE', 'SKIN-PUNCTATE',
       'SKIN-PUNCTATE-ROUND_AND_REGULAR', 'MILK_OF_CALCIUM',
       'PLEOMORPHIC-PLEOMORPHIC', 'SKIN-COARSE-ROUND_AND_REGULAR',
       'AMORPHOUS-ROUND_AND_REGULAR', 'ROUND_AND_REGULAR-PLEOMORPHIC',
       'ROUND_AND_

In [40]:
df.nunique(axis=0).sum()
df.nunique(axis=1)

6095

0       12
1       12
2       12
3       12
4       12
        ..
1867    14
1868    14
1869    14
1870    13
1871    13
Length: 1872, dtype: int64

## Completeness

In [15]:
complete = df.count().to_frame()
null = df.isnull().sum()
c = pd.concat([complete, null], axis=1)
c.columns = ['Complete', 'Null']
c['N'] = c['Complete'] + c['Null']
c['Completeness'] = c['Complete'] / c['N']
c.drop(columns='Null', inplace=True)
c = c[['N', 'Complete', 'Completeness']]
c
sc = c[['N', 'Complete']].sum().to_frame().T
sc
sc['Completeness'] = sc['Complete'] / sc['N']
sc


,N,Complete,Completeness
patient_id,1872,1872,1.00
breast_density,1872,1872,1.00
left_or_right_breast,1872,1872,1.00
image_view,1872,1872,1.00
abnormality_id,1872,1872,1.00
abnormality_type,1872,1872,1.00
calc_type,1872,1848,0.99
calc_distribution,1872,1433,0.77
assessment,1872,1872,1.00
pathology,1872,1872,1.00


,N,Complete
0,28080,27617


,N,Complete,Completeness
0,28080,27617,0.98


In [16]:
vars = ['calc_type', 'calc_distribution']
vars = 'calc_type'
#df.loc[df[vars].isnull().any(axis=1)]
df.loc[df[vars].isnull()]

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,dataset,image_series_uid,roi_mask_series_uid,cropped_image_series_uid
4,P_00008,1,LEFT,CC,1,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.406725628213826290127...,1.3.6.1.4.1.9590.100.1.2.281397494612871934937...,1.3.6.1.4.1.9590.100.1.2.281397494612871934937...
5,P_00008,1,LEFT,CC,2,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.406725628213826290127...,1.3.6.1.4.1.9590.100.1.2.202418111353862740689...,1.3.6.1.4.1.9590.100.1.2.202418111353862740689...
6,P_00008,1,LEFT,CC,3,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.406725628213826290127...,1.3.6.1.4.1.9590.100.1.2.358964325411863896506...,1.3.6.1.4.1.9590.100.1.2.358964325411863896506...
7,P_00008,1,LEFT,MLO,1,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.147846893112398086048...,1.3.6.1.4.1.9590.100.1.2.812167235138269533227...,1.3.6.1.4.1.9590.100.1.2.812167235138269533227...
8,P_00008,1,LEFT,MLO,2,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.147846893112398086048...,1.3.6.1.4.1.9590.100.1.2.101644730412034535338...,1.3.6.1.4.1.9590.100.1.2.101644730412034535338...
9,P_00008,1,LEFT,MLO,3,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.147846893112398086048...,1.3.6.1.4.1.9590.100.1.2.416588201163928413438...,1.3.6.1.4.1.9590.100.1.2.416588201163928413438...
10,P_00008,1,RIGHT,CC,1,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.226230437127681311097...,1.3.6.1.4.1.9590.100.1.2.381673661212780323135...,1.3.6.1.4.1.9590.100.1.2.381673661212780323135...
11,P_00008,1,RIGHT,CC,2,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.226230437127681311097...,1.3.6.1.4.1.9590.100.1.2.402144517512294257725...,1.3.6.1.4.1.9590.100.1.2.402144517512294257725...
12,P_00008,1,RIGHT,CC,3,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.226230437127681311097...,1.3.6.1.4.1.9590.100.1.2.471809893127172992126...,1.3.6.1.4.1.9590.100.1.2.471809893127172992126...
13,P_00008,1,RIGHT,CC,4,calcification,NaN,REGIONAL,2,BENIGN,3,train,1.3.6.1.4.1.9590.100.1.2.226230437127681311097...,1.3.6.1.4.1.9590.100.1.2.140640288118004014393...,1.3.6.1.4.1.9590.100.1.2.140640288118004014393...


In [17]:
n = df.count() + df.isnull().sum()
n

patient_id                  1872
breast_density              1872
left_or_right_breast        1872
image_view                  1872
abnormality_id              1872
abnormality_type            1872
calc_type                   1872
calc_distribution           1872
assessment                  1872
pathology                   1872
subtlety                    1872
dataset                     1872
image_series_uid            1872
roi_mask_series_uid         1872
cropped_image_series_uid    1872
dtype: int64

In [18]:
c = df.notnull().sum()
c

patient_id                  1872
breast_density              1872
left_or_right_breast        1872
image_view                  1872
abnormality_id              1872
abnormality_type            1872
calc_type                   1848
calc_distribution           1433
assessment                  1872
pathology                   1872
subtlety                    1872
dataset                     1872
image_series_uid            1872
roi_mask_series_uid         1872
cropped_image_series_uid    1872
dtype: int64

In [19]:
cn = c/n
cn

patient_id                 1.00
breast_density             1.00
left_or_right_breast       1.00
image_view                 1.00
abnormality_id             1.00
abnormality_type           1.00
calc_type                  0.99
calc_distribution          0.77
assessment                 1.00
pathology                  1.00
subtlety                   1.00
dataset                    1.00
image_series_uid           1.00
roi_mask_series_uid        1.00
cropped_image_series_uid   1.00
dtype: float64

In [20]:
dfc = pd.concat([n,c,cn], axis=1)
dfc.columns = ['N', 'Complete', 'Completeness']
dfc

,N,Complete,Completeness
patient_id,1872,1872,1.00
breast_density,1872,1872,1.00
left_or_right_breast,1872,1872,1.00
image_view,1872,1872,1.00
abnormality_id,1872,1872,1.00
abnormality_type,1872,1872,1.00
calc_type,1872,1848,0.99
calc_distribution,1872,1433,0.77
assessment,1872,1872,1.00
pathology,1872,1872,1.00


In [21]:
c.sum() / n.sum()

0.983511396011396

In [22]:
u = df.nunique()
u

patient_id                   753
breast_density                 5
left_or_right_breast           2
image_view                     2
abnormality_id                 7
abnormality_type               1
calc_type                     47
calc_distribution              9
assessment                     5
pathology                      2
subtlety                       5
dataset                        2
image_series_uid            1511
roi_mask_series_uid         1872
cropped_image_series_uid    1872
dtype: int64

In [23]:
df.nunique().sum()

6095

In [24]:
u / n

patient_id                 0.40
breast_density             0.00
left_or_right_breast       0.00
image_view                 0.00
abnormality_id             0.00
abnormality_type           0.00
calc_type                  0.03
calc_distribution          0.00
assessment                 0.00
pathology                  0.00
subtlety                   0.00
dataset                    0.00
image_series_uid           0.81
roi_mask_series_uid        1.00
cropped_image_series_uid   1.00
dtype: float64

In [25]:
d = len(df) + (len(df)) - len(df.drop_duplicates())
d

1872

In [26]:
len(df[df.isnull()].any(axis=1))

1872

In [27]:
values = [0,1,2,3,4]
df['breast_density'].isin(values).sum() 

1872

In [28]:
print(os.getcwd())

/home/john/projects/bcd


In [29]:
uid = "1.3.6.1.4.1.9590.100.1.2.40629582313887326118580579092807259508"
len(uid)


63

In [30]:
df.notnull().all(axis=1).sum()

1409

In [36]:
d= {'n': 1254, 'valid': np.arange(5)}
df = pd.DataFrame(d)
df.sum(axis=0).to_frame()

,0
n,6270
valid,10
